In [ ]:
!pip install streamlit
#!pip install --upgrade protobuf

In [2]:
!pip install protobuf==3.20.*

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 5.5 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.27.4
    Uninstalling protobuf-5.27.4:
      Successfully uninstalled protobuf-5.27.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apache-beam 2.46.0 requires grpcio!=1.48.0,<2,>=1.33.1, but you have grpcio 1.48.0 which is incompatible.
google-api-python-client 1.8.0 requires google-api-core<2dev,>=1.13.0, but you have google-api-core 2.11.1 which is incompatible.
google-cloud-pubsub 2.21.4 requires grpcio<2.0dev,>=1.51.3, but you have grpcio 1.48.0 which is incompatible.
tensorflow 2.11.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.3 which is incompatible.
tensorflow-serving-api 2.11.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.3 which is incompatible.


In [1]:
from google.cloud import bigquery
from google.cloud.aiplatform_v1.types import NearestNeighborQuery
from vertexai.resources.preview import (FeatureOnlineStore, FeatureView,
                                        FeatureViewBigQuerySource)
from vertexai.resources.preview.feature_store import utils

#set project info
PROJECT = !gcloud config get-value project
PROJECT_ID = PROJECT[0]
REGION = "us-central1" 
 
#Verify that the FeatureView instance is created by getting the feature view.
FEATURE_ONLINE_STORE_ID = "nine_quality_test_multimodal_featurestore"  # @param {type: "string"}
FEATURE_VIEW_ID = "multimodal_feature_view_nine_quality_test"  # @param {type: "string"}
MM_nine_fv=FeatureView(
    FEATURE_VIEW_ID, feature_online_store_id=FEATURE_ONLINE_STORE_ID
) 
FEATURE_ONLINE_STORE_ID = "nine_quality_test_content_featurestore"  # @param {type: "string"}
FEATURE_VIEW_ID = "content_feature_view_nine_quality_test"  # @param {type: "string"}
content_nine_fv=FeatureView(
    FEATURE_VIEW_ID, feature_online_store_id=FEATURE_ONLINE_STORE_ID
) 

2024-08-28 11:48:05.438803: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-28 11:48:06.597680: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2024-08-28 11:48:06.597833: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/loca

In [2]:
def get_response(feature_vector,converted_query_to_embedding,n_neighbor=10):
    result=feature_vector.search(
        embedding_value=converted_query_to_embedding,
        neighbor_count=n_neighbor,
        #string_filters=[country_filter],#for multimodal embedding this can be set to None, unless having a description column
        return_full_entity=True,  # returning entities with metadata
    )
    result=result.to_dict()
    return result
 
def get_neighbours(neighbours,embedding_type):
    nearest_neighbours=[]
    for neighbour in neighbours['neighbors']:
        nearest_neighbour={}
        nearest_neighbour['entity_id']=neighbour['entity_id']
        nearest_neighbour['distance']=neighbour['distance']

        for feature in neighbour['entity_key_values']['key_values']['features']:
            if 'value' in feature:
                if type(list(feature['value'].values())[0]) is dict:
                    nearest_neighbour[feature['name']]=[]#list(list(feature['value'].values())[0].values())[0]             
                else:
                    nearest_neighbour[feature['name']]=list(feature['value'].values())[0]             
            else :
                nearest_neighbour[feature['name']]=None
            nearest_neighbour['embedding_type']=embedding_type

        nearest_neighbours.append(nearest_neighbour)
    return nearest_neighbours
    

In [7]:
import streamlit as st
#from predict_request_gapic import *
import json
#import prediction client
from Embeddings.EmbeddingPredictionCLS import EmbeddingPredictionClient
neighbor_count=10

st.set_page_config(
     layout="wide",
     page_title="JS Lab",
     page_icon="https://api.dicebear.com/5.x/bottts-neutral/svg?seed=gptLAb"
)

# Sidebar
st.sidebar.header("About")
st.sidebar.markdown(
    "9News Search Engine POC"
)

st.title("9News Search Engine: Semantic Searches for Images and Videos")
st.subheader("Implemented by Intelia")


#client to access multimodal-embeddings model to convert text to embeddings
client = EmbeddingPredictionClient(project=PROJECT_ID)


scopes = ["https://www.googleapis.com/auth/cloud-platform"]
sa_file_path = "serious-hall-371508-b584a49b4817.json"
##credentials = service_account.Credentials.from_service_account_file(sa_file_path, scopes=scopes)
##client_options = { "api_endpoint": "1256643097.asia-southeast1-712487506603.vdb.vertexai.goog"}

#client to access GCS bucket
##storage_client = storage.Client(credentials=credentials)
##bucket = storage_client.bucket("js-multimodal-embeddings")

 

allResults=[]

search_term = 'I am searching for ' + st.text_input('Search: ')
if search_term !='I am searching for ':
    converted_query_to_mm_embedding = client.get_multimodal_embedding(text=search_term).text_embedding
    converted_query_to_text_embedding = client.get_text_embedding(text=search_term).text_embedding
    nearest_neighbours=[]
    
    result_mm=get_response(MM_nine_fv,converted_query_to_mm_embedding,neighbor_count)
    result_content=get_response(content_nine_fv,converted_query_to_text_embedding,neighbor_count)
   
    mm_nearest_neighbours=get_neighbours(result_mm,'multimodal')
    content_nearest_neighbours=get_neighbours(result_content,'content')
    allResults=mm_nearest_neighbours+content_nearest_neighbours
        
if len(allResults)>=1:
 st.write("")
 st.write("These are the most relevant results matching your search query:")
 st.image(allResults, width=200)
elif search_term =='I am searching for ':
 st.write("Please type in a search query above")
else:
 st.write("Sorry! There are no images matching your query. Please try again.")

2024-08-28 11:30:47.488 Session state does not function when running a script without `streamlit run`


In [13]:
print(converted_query_to_text_embedding)

[0.013925430364906788, -0.030013956129550934, -0.012476854957640171, -0.02868626080453396, 0.0007188087911345065, 0.028315065428614616, 0.01622144505381584, 0.0023571078199893236, -0.009538206271827221, 0.02664007991552353, -0.004647109191864729, -0.027522103860974312, -0.02407633699476719, -0.025457285344600677, 0.019135231152176857, -0.02721548266708851, 0.02384776808321476, 0.0054380157962441444, 0.053539860993623734, -0.046579618006944656, -0.011654962785542011, 0.015526391565799713, 0.00038028223207220435, -0.019228210672736168, 0.007561243139207363, -0.02677755616605282, 0.04680021479725838, -0.0868605226278305, -0.033086325973272324, 0.051227014511823654, -0.07389315962791443, 0.014334295876324177, -0.038420822471380234, 0.018949151039123535, 0.01672111451625824, -0.06482402980327606, -0.0027009453624486923, -0.03352271392941475, 0.01752518117427826, 0.03261937201023102, 0.012928116135299206, -0.009585859254002571, -0.018772678449749947, -0.025923749431967735, 0.0758854076266288

In [5]:
import json
#import prediction client
from Embeddings.EmbeddingPredictionCLS import EmbeddingPredictionClient
neighbor_count=10
client = EmbeddingPredictionClient(project=PROJECT_ID)


In [6]:
search_term='president of USA'

converted_query_to_mm_embedding = client.get_multimodal_embedding(text=search_term).text_embedding
converted_query_to_text_embedding = client.get_text_embedding(text=search_term).text_embedding
nearest_neighbours=[]
    
result_mm=get_response(MM_nine_fv,converted_query_to_mm_embedding,neighbor_count)
result_content=get_response(content_nine_fv,converted_query_to_text_embedding,neighbor_count)
   
mm_nearest_neighbours=get_neighbours(result_mm,'multimodal')
content_nearest_neighbours=get_neighbours(result_content,'content')
allResults=mm_nearest_neighbours+content_nearest_neighbours

[fields {
  key: "text"
  value {
    string_value: "president of USA"
  }
}
]
Public endpoint for the optimized online store nine_quality_test_multimodal_featurestore is 5438105346142896128.us-central1-494586852359.featurestore.vertexai.goog
Public endpoint for the optimized online store nine_quality_test_content_featurestore is 7347631588147986432.us-central1-494586852359.featurestore.vertexai.goog


In [7]:
allResults

[{'entity_id': '60MI24_1_A_HBB.mp4|2400|2416',
  'distance': -0.1292482316493988,
  'end_offset_sec_embedding': '2416',
  'embedding_type': 'multimodal',
  'media_type': 'video',
  'start_offset_sec_embedding': '2400',
  'multimodal_embedding': [],
  'path': 'gs://raw_nine_files/60MI24_1_A_HBB.mp4'},
 {'entity_id': '60MI24_1_A_HBB.mp4|2416|2432',
  'distance': -0.12086805701255798,
  'end_offset_sec_embedding': '2432',
  'embedding_type': 'multimodal',
  'media_type': 'video',
  'start_offset_sec_embedding': '2416',
  'multimodal_embedding': [],
  'path': 'gs://raw_nine_files/60MI24_1_A_HBB.mp4'},
 {'entity_id': '60MI24_1_A_HBB.mp4|1936|1952',
  'distance': -0.1192726120352745,
  'end_offset_sec_embedding': '1952',
  'embedding_type': 'multimodal',
  'media_type': 'video',
  'start_offset_sec_embedding': '1936',
  'multimodal_embedding': [],
  'path': 'gs://raw_nine_files/60MI24_1_A_HBB.mp4'},
 {'entity_id': '60MI24_1_A_HBB.mp4|2392|2400',
  'distance': -0.11390428990125656,
  'end_off